In [10]:
import pandas as pd
import numpy as np
root = '../dataset/hi'

# Load dataset
hiv = pd.read_csv(f'{root}/hiv-charge-CS2_36.csv')
hii = pd.read_csv(f'{root}/hii-charge-CS2_36.csv')

In [2]:
# Unisci i due dataset utilizzando la colonna "cycle"
df = pd.merge(hiv, hii, on="cycle")

# Rimuovi la colonna "cycle" dal DataFrame risultante
df = df.drop(columns=["cycle"], inplace=False)

# Visualizza il DataFrame risultante
print(df)

         hi_v      hi_i
0    0.139086  0.082556
1    6.067685  0.105748
2    6.071450  0.104493
3    6.094198  0.101743
4    6.041353  0.098991
..        ...       ...
955  5.715144  0.122793
956  5.710389  0.122829
957  5.732181  0.114557
958  5.709578  0.112152
959  5.662337  0.117791

[960 rows x 2 columns]


In [3]:
# Split into train and test sets
train_size = int(len(df) * 0.8)
train_data, test_data = df[:train_size], df[train_size:]

In [5]:
from sklearn.preprocessing import StandardScaler

# Istanzia lo StandardScaler
scaler = StandardScaler()

# Standardizza i dati
scaled_data = scaler.fit_transform(df)

# Crea un nuovo DataFrame con i dati standardizzati
df_standardized = pd.DataFrame(scaled_data, columns=df.columns)


In [6]:
df

,hi_v,hi_i
0,0.139086,0.082556
1,6.067685,0.105748
2,6.071450,0.104493
3,6.094198,0.101743
4,6.041353,0.098991
...,...,...
955,5.715144,0.122793
956,5.710389,0.122829
957,5.732181,0.114557
958,5.709578,0.112152


In [22]:
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        seq = data[['hi_v', 'hi_i']].iloc[i:i+seq_length].values
        target = data['hi_i'].iloc[i+seq_length]
        X.append(seq)
        y.append(target)
    return np.array(X), np.array(y)


In [24]:
# Define the sequence length
seq_length = 10

# Create sequences for training set
X_train, y_train = create_sequences(train_data, seq_length)

# Print the shape of X_train before reshaping
print("Shape of X_train before reshaping:", X_train.shape)

# Reshape input data for training set
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 2))

# Print the shape of X_train after reshaping
print("Shape of X_train after reshaping:", X_train.shape)

# Create sequences for testing set
X_test, y_test = create_sequences(test_data, seq_length)

# Print the shape of X_test before reshaping
print("Shape of X_test before reshaping:", X_test.shape)

# Reshape input data for testing set
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 2))

# Print the shape of X_test after reshaping
print("Shape of X_test after reshaping:", X_test.shape)


Shape of X_train before reshaping: (758, 10, 2)
Shape of X_train after reshaping: (758, 10, 2)
Shape of X_test before reshaping: (182, 10, 2)
Shape of X_test after reshaping: (182, 10, 2)
